In [3]:
import requests 
import pandas as pd
import seaborn as sns
import numpy as np 
from datetime import datetime
from sqlalchemy import create_engine
from keys import *

In [4]:
global  bearer, full_directory_lst
full_directory_lst = []

url = f'https://id.twitch.tv/oauth2/token?client_id={my_client_id}&client_secret={my_client_secret}&grant_type=client_credentials'
x = requests.post(url)
bearer = x.text[17:47]


def get_directory():
    url = 'https://api.twitch.tv/helix/games/top'
    headers = { "Client-ID": f'{my_client_id}','Authorization': f'Bearer {bearer}',  'Accept': "application/vnd.twitchtv.v5+json", }
    resp = requests.get(url, headers = headers).json()
    for game in range(10):
        get_directory_viewers(resp['data'][game]['id'], resp['data'][game]['name'])


In [5]:
def get_directory_viewers(game_id, game_name):
    url= f'https://api.twitch.tv/helix/streams?first=100&game_id={game_id}'
    headers = { "Client-ID": f'{my_client_id}','Authorization': f'Bearer {bearer}',  'Accept': "application/vnd.twitchtv.v5+json", }
    resp = requests.get(url, headers = headers).json()
    pig=resp['pagination']['cursor']

    total = 0


    while len(resp['data']) > 1:
        for x in range(len(resp['data'])):
            total += resp['data'][x]['viewer_count']


        url= f'https://api.twitch.tv/helix/streams?after={pig}&first=100&game_id={game_id}'
        headers = { "Client-ID": f'{my_client_id}','Authorization': f'Bearer {bearer}',  'Accept': "application/vnd.twitchtv.v5+json", }
        resp = requests.get(url, headers = headers).json()
        try:
            pig=resp['pagination']['cursor']
        except:
            pig=''
    
    full_directory_lst.append([game_name, total])


In [6]:
def make_twitch_df(full_directory_lst):
    twitch_df = pd.DataFrame(columns=['Date', 'Time', 'Directory Name', 'Viewer Count'])
    
    date = str(datetime.now()).split(' ')[0]
    time = str(datetime.now()).split(' ')[1].split('.')[0]

    for item in full_directory_lst:
        twitch_df = twitch_df.append({'Date' : date, 'Time': time, 'Directory Name': item[0], 'Viewer Count' : item[1]},ignore_index=True )
    return twitch_df

In [7]:
def create_db():
    global engine
    # create sqlite engine
    engine = create_engine('sqlite:///twitch.db', echo=True)
    # create connection to engine
    conn = engine.connect()

    return conn

In [8]:
def add_to_db(twitch_df, conn):
    sqlite_table = "Twitch_Directory"
    twitch_df.to_sql(sqlite_table, conn, if_exists='append')


In [9]:
get_directory()
twitch_df = make_twitch_df(full_directory_lst)
conn = create_db()
add_to_db(twitch_df, conn)

{'data': [{'id': '509658', 'name': 'Just Chatting', 'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/509658-{width}x{height}.jpg'}, {'id': '32982', 'name': 'Grand Theft Auto V', 'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/32982_IGDB-{width}x{height}.jpg'}, {'id': '21779', 'name': 'League of Legends', 'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/21779-{width}x{height}.jpg'}, {'id': '516575', 'name': 'VALORANT', 'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/516575-{width}x{height}.jpg'}, {'id': '33214', 'name': 'Fortnite', 'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/33214-{width}x{height}.jpg'}, {'id': '511224', 'name': 'Apex Legends', 'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/511224-{width}x{height}.jpg'}, {'id': '498983', 'name': 'Holdfast: Nations At War', 'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/498983_IGDB-{width}x{height}.jpg'}, {'id': '498566', 'name': 'Slots', 'box_art_url': 'https://static-cdn.jtvnw

In [10]:
sql = """
SELECT *
FROM Twitch_Directory
"""
# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2022-05-04 18:55:01,541 INFO sqlalchemy.engine.base.OptionEngine 
SELECT *
FROM Twitch_Directory

2022-05-04 18:55:01,542 INFO sqlalchemy.engine.base.OptionEngine ()


,index,Date,Time,Directory Name,Viewer Count
0,0,2022-01-31,10:45:52,Just Chatting,390431
1,1,2022-01-31,10:45:52,Grand Theft Auto V,239160
2,2,2022-01-31,10:45:52,League of Legends,219827
3,3,2022-01-31,10:45:52,VALORANT,137060
4,4,2022-01-31,10:45:52,Fortnite,115131
